# 3rd Home Assignment

Objective: make the best possible machine learning supervised model for inferring molecular activity

In [ ]:
import pickle
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, explained_variance_score, mean_squared_error, mean_absolute_percentage_error
from scipy.stats import pearsonr
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, cross_val_score
from sklearn import tree, metrics
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

## Data Treatment

In [ ]:
X_train, X_ivs, y_train, col_names = pickle.load(open("drd2_data.pickle", "rb"))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [ ]:
print(X_train, X_test, y_train, y_test)
dimensions = np.shape(X_train)
rows, columns = dimensions
print(rows, columns)


[[454.17870969 454.556        9.         ...   0.           0.
    0.        ]
 [341.19550624 341.445        1.         ...   0.           0.
    0.        ]
 [321.17287898 321.42         3.         ...   1.           0.
    0.        ]
 ...
 [319.19361442 319.448        2.         ...   0.           0.
    0.        ]
 [433.19771173 433.474        5.         ...   0.           0.
    0.        ]
 [569.31546087 569.753        6.         ...   0.           0.
    0.        ]] [[1134.50764389 1136.226        14.         ...    0.
     0.            0.        ]
 [ 378.24196158  378.52          5.         ...    0.
     0.            0.        ]
 [ 487.2558953   487.57          6.         ...    0.
     0.            0.        ]
 ...
 [ 453.1641612   454.039         4.         ...    0.
     0.            0.        ]
 [ 406.20049069  406.486         7.         ...    0.
     0.            0.        ]
 [ 326.12982429  326.831         4.         ...    0.
     0.            0.        ]] [0. 

## Feature Selection

In [ ]:
def naif_model_testingR(X_train, X_test, y_train, y_test):
    #test 3 approaches and print out the results
    
    rfr= RandomForestRegressor(n_estimators=100)
    rfr.fit(X_train, y_train)

    dtr= DecisionTreeRegressor(max_depth=5)
    dtr.fit(X_train, y_train)

    lmr=LinearRegression()
    lmr.fit(X_train, y_train)

    rf_preds=rfr.predict(X_test)
    dt_preds=dtr.predict(X_test)
    lr_preds=lmr.predict(X_test)

    print("RVE RFs: %7.4f" % explained_variance_score(y_test, rf_preds))
    print("RVE DTs: %7.4f" % explained_variance_score(y_test, dt_preds))
    print("RVE LRs: %7.4f" % explained_variance_score(y_test, lr_preds))

In [ ]:
from sklearn.feature_selection import SelectFromModel
N,M=X_train.shape
rfr=RandomForestRegressor(random_state=0)
sel = SelectFromModel(estimator=rfr, threshold=.001) #Change the threshold! See what happens!
sel.fit(X_train, y_train)

print("Importances: ", sel.estimator_.feature_importances_)

print("Default threshold: ", sel.threshold_)

features=sel.get_support()
Features_selected =np.arange(M)[features]
print("The features selected are columns: ", Features_selected)

nX_train=sel.transform(X_train)
nX_test=sel.transform(X_test)
nX_ivs=sel.transform(X_ivs)
naif_model_testingR(nX_train, nX_test, y_train, y_test)


Importances:  [3.85716938e-03 4.23060158e-03 2.48467172e-03 ... 8.64906260e-05
 4.00366427e-04 1.68565580e-04]
Default threshold:  0.001
The features selected are columns:  [   0    1    2    3    4    5    6    7    8    9   10   11   12   13
   15   16   17   19   22   23   24   25   26   27   28   29   30   31
   32   33   34   35   36   37   38   39   40   41   42   74   83   96
  103  122  154  180  186  216  246  287  303  316  336  347  348  353
  362  364  386  390  442  444  472  494  525  546  551  561  570  679
  695  765  815  819  839  853  871  879  886  889  891  909  916  924
  939  950  956  959  965  966  981  982 1013 1020 1029 1054 1063 1075
 1086 1087 1121 1147 1161 1166 1167 1168 1176 1214 1219 1248 1284 1285
 1303 1319 1351 1397 1440 1454 1499 1527 1538 1540 1575 1598 1604 1649
 1667 1687 1706 1709 1766 1778 1782 1793 1815 1841 1868 1893 1916 1921
 1922 1925 1934 1968 2069 2080]
RVE RFs:  0.5866
RVE DTs:  0.2538
RVE LRs:  0.3920


In [ ]:
#Standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(nX_train)
X_test_scaled = scaler.transform(nX_test)
X_ivs_scaled = scaler.transform(nX_ivs)

## Defining Statistical Functions

In [ ]:
def printRegStatistics(truth, preds):
    #corr, pval = pearsonr(truth, preds)
    print('The R^2 is: ', r2_score(truth, preds))
    print('The MSE is: ', mean_squared_error(truth, preds)*100,'%')


# Define cross-validation function
def cross_val_rmse(model, X, y, cv=5):
    cv_scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error')
    cv_rmse_scores = np.sqrt(-cv_scores)
    mean_rmse = np.mean(cv_rmse_scores)
    return cv_rmse_scores, mean_rmse

## Cross Validation

In [ ]:
def cross_val_rmse(model, X, y, cv=5):
    cv_scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error')
    cv_rmse_scores = np.sqrt(-cv_scores)
    mean_rmse = np.mean(cv_rmse_scores)
    return cv_rmse_scores, mean_rmse

## Regression Models

### Linear Regression

In [ ]:
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train)
y_pred_linear = linear_model.predict(X_test_scaled)
linear_cv_scores, linear_mean_rmse = cross_val_rmse(linear_model, X_train_scaled, y_train)

print(f'Linear Regression - Cross-Validation RMSE Scores: {linear_cv_scores}')
print(f'Mean RMSE: {linear_mean_rmse}')
printRegStatistics(y_test, y_pred_linear)

Linear Regression - Cross-Validation RMSE Scores: [0.20588953 0.20943802 0.21312064 0.20728813 0.20870562]
Mean RMSE: 0.20888839010875407
The R^2 is:  0.3918275168012867
The MSE is:  4.611074160599747 %


### K Nearest Neighbors Regressor

In [ ]:
knn_model = KNeighborsRegressor()
knn_param_grid = {'n_neighbors': [3, 5, 7, 9]}
knn_grid_search = GridSearchCV(knn_model, knn_param_grid, cv=5, scoring='neg_mean_squared_error')
knn_grid_search.fit(X_train_scaled, y_train)

knn_model_best = knn_grid_search.best_estimator_
knn_final = knn_model_best.fit(X_train_scaled, y_train)
y_pred_knn = knn_final.predict(X_test_scaled)
knn_cv_scores, knn_mean_rmse = cross_val_rmse(knn_model, X_train_scaled, y_train)

print (knn_grid_search.best_params_)
print(f'K Nearest Neighbors - Cross-Validation RMSE Scores: {knn_cv_scores}')
print(f'Mean RMSE: {knn_mean_rmse}')
printRegStatistics(y_test, y_pred_knn)

{'n_neighbors': 5}
K Nearest Neighbors - Cross-Validation RMSE Scores: [0.17541644 0.18742583 0.18457315 0.18074196 0.18346625]
Mean RMSE: 0.18232472602285404
The R^2 is:  0.5554294662222707
The MSE is:  3.370668284899578 %


### Decision Tree Regressor

In [ ]:
dt_model = DecisionTreeRegressor(max_depth=None, random_state=42)
dt_model.fit(X_train_scaled, y_train)
y_pred_dt = dt_model.predict(X_test_scaled)
dt_cv_scores, dt_mean_rmse = cross_val_rmse(dt_model, X_train_scaled, y_train)

print(f'Decision Tree Regressor - Cross-Validation RMSE Scores: {dt_cv_scores}')
print(f'Mean RMSE: {dt_mean_rmse}')
printRegStatistics(y_test, y_pred_dt)

Decision Tree Regressor - Cross-Validation RMSE Scores: [0.2438934  0.25423496 0.25382286 0.23607878 0.23985519]
Mean RMSE: 0.24557703745018955
The R^2 is:  0.265902667538375
The MSE is:  5.565817814175915 %


### Random Forrest

In [0]:
rf_model = RandomForestRegressor()
rf_param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf_grid_search = GridSearchCV(rf_model, rf_param_grid, cv=5, scoring='neg_mean_squared_error')
rf_grid_search.fit(X_train_scaled, y_train)

rf_model_best = rf_grid_search.best_estimator_
rf_final = rf_model_best.fit(X_train_scaled, y_train)
y_pred_rf = rf_model_best.predict(X_test_scaled)
rf_cv_scores, rf_mean_rmse = cross_val_rmse(rf_model, X_train_scaled, y_train)

print (rf_grid_search.best_params_)
print(f'Random Forest - Cross-Validation RMSE Scores: {rf_cv_scores}')
print(f'Mean RMSE: {rf_mean_rmse}')
printRegStatistics(y_test, y_pred_rf)

{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}
Random Forest - Cross-Validation RMSE Scores: [0.16665308 0.18026983 0.1795669  0.17309693 0.173982  ]
Mean RMSE: 0.17471374601884354
The R^2 is:  0.5930833242899138
The MSE is:  3.0851822808807654 %


### Neural Network

In [ ]:
nn_model = MLPRegressor(max_iter=500)
nn_param_grid = {'hidden_layer_sizes': [(50,), (100,), (200,)]}
nn_grid_search = GridSearchCV(nn_model, nn_param_grid, cv=5, scoring='neg_mean_squared_error')
nn_grid_search.fit(X_train_scaled, y_train)
nn_model_best = nn_grid_search.best_estimator_
y_pred_nn = nn_model_best.predict(X_test_scaled)
nn_cv_scores, nn_mean_rmse = cross_val_rmse(nn_model, X_train_scaled, y_train)

print(f'Neural Network - Cross-Validation RMSE Scores: {nn_cv_scores}')
print(f'Mean RMSE: {nn_mean_rmse}')
printRegStatistics(y_test, y_pred_nn)

Neural Network - Cross-Validation RMSE Scores: [0.24792469 0.23975806 0.23386512 0.22147861 0.24564862]
Mean RMSE: 0.23773501842027372
The R^2 is:  0.34464026768148515
The MSE is:  4.968840930943757 %


## Predictions

In [ ]:
# Apply post-processing to ensure values are between 0 and 1
def constrain_predictions(predictions):
    return np.clip(predictions, 0, 1)

# Post-process predictions
y_pred_linear_constrained = constrain_predictions(y_pred_linear)
y_pred_knn_constrained = constrain_predictions(y_pred_knn)
y_pred_dt_constrained = constrain_predictions(y_pred_dt)
y_pred_rf_constrained = constrain_predictions(y_pred_rf)
y_pred_nn_constrained = constrain_predictions(y_pred_nn)

# Evaluate the models
mse_linear = mean_squared_error(y_test, y_pred_linear_constrained)
mse_knn = mean_squared_error(y_test, y_pred_knn_constrained)
mse_dt = mean_squared_error(y_test, y_pred_dt_constrained)
mse_rf = mean_squared_error(y_test, y_pred_rf_constrained)
mse_nn = mean_squared_error(y_test, y_pred_nn_constrained)

print(f'Random Forest - Mean Squared Error on Test Set: {mse_rf}')
print(f'Neural Network - Mean Squared Error on Test Set: {mse_nn}')
print(f'Linear Regression - Mean Squared Error on Test Set: {mse_linear}')
print(f'K Nearest Neighbors - Mean Squared Error on Test Set: {mse_knn}')
print(f'Decision Tree - Mean Squared Error on Test Set: {mse_dt}')

# Make predictions on the X_ivs set and apply post-processing
y_ivs_pred_linear = constrain_predictions(linear_model.predict(X_ivs_scaled))
y_ivs_pred_knn = constrain_predictions(knn_final.predict(X_ivs_scaled))
y_ivs_pred_dt = constrain_predictions(dt_model.predict(X_ivs_scaled))
y_ivs_pred_rf = constrain_predictions(rf_final.predict(X_ivs_scaled))
y_ivs_pred_nn = constrain_predictions(nn_model_best.predict(X_ivs_scaled))

np.savetxt("linear_predictions.txt", y_ivs_pred_linear, delimiter='\n')
np.savetxt("knn_predictions.txt", y_ivs_pred_knn, delimiter='\n')
np.savetxt("dt_predictions.txt", y_ivs_pred_dt, delimiter='\n')
np.savetxt("rf_predictions.txt", y_ivs_pred_rf, delimiter='\n')
np.savetxt("nn_predictions.txt", y_ivs_pred_nn, delimiter='\n')

Random Forest - Mean Squared Error on Test Set: 0.030851822808807654
Neural Network - Mean Squared Error on Test Set: 0.0463499509852415
Linear Regression - Mean Squared Error on Test Set: 0.04609703407415841
K Nearest Neighbors - Mean Squared Error on Test Set: 0.03370668284899578
Decision Tree - Mean Squared Error on Test Set: 0.05565817814175915


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4dab6db1-1065-4405-bcab-94ff6253bbc5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>